# Práctica 6.4. Transfer Learning con Transformers de Hugging Face (🤗)

Esta práctica está basada en la cuarta sesión del fantástico curso [NLP de 0 a 100](https://somosnlp.org/nlp-de-cero-a-cien/sesion-04), organizado por [NLP en ES](https://somosnlp.org/), Spain AI y Hugging Face, e impartido por Lewis Tunstall.

Aquí veremos cómo usar la API de Hugging Face, descargar un dataset, un modelo pre-entrenado (RoBERTa en español) y hacer fine-tuning para clasificar las reseñas de Amazon.

## 1. Instalación del entorno Hugging Face

Si estás ejecutando este notebook en Google Colab, ejecuta la siguiente celda para instalar la biblioteca de transformers que necesitamos. Esto instalará el entorno de Hugging Face (también conocido con el emoticono 🤗). Si estás en un entorno en tu equipo local, lanza la instrucción en una Terminal. En realidad vamos a volver a usar la librería `datasets`, que hemos cubierto en una práctica anterior.

In [1]:
# Si el entrenamiento te pide una clave para WANDB, ejecuta estas líneas
#import os
#os.environ["WANDB_DISABLED"] = "true"
import os

# Desactiva el paralelismo de los tokenizadores para evitar conflictos
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["USE_TF"] = "0"
os.environ["USE_TORCH"] = "1"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Si no lo has hecho aún, instala las librerías necesarias (transformers, datasets y evaluate):

In [2]:
#!pip install transformers datasets evaluate

In [ ]:
# Es posible que sea necesario instalar también estos paquetes
#!pip install sentencepiece 
#!pip install safetensors

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 2. Carga y exploración del dataset

Utilizaremos Hugging Face 🤗 Datasets para cargar y procesar nuestro conjunto de datos. 




En la siguiente celda vamos a descargar el dataset de amazon reviews, en español. Esto descarga un objeto que es un diccionario, donde las claves son los subconjuntos del dataset (train, val y train). También podrás ver las características (columnas) del dataset, y el número de ejemplos (filas).

*Nota: El dataset oficial ha sido retirado por Amazon, y solo está disponible en [Kaggle](https://www.kaggle.com/datasets/mexwell/amazon-reviews-multi). Por suerte, aún queda una copia, que es la que vamos a utilizar. Sin embargo, si falla, puedes transformar este dataset de Kaggle a Dataset de HuggingFace, con [estos pasos](https://github.com/huggingface/datasets/issues/6109#issuecomment-2189011329).*

In [3]:
# importamos la librería de datasets
from datasets import load_dataset

# descargamos el dataset de amazon reviews en español
#dataset = load_dataset("amazon_reviews_multi", "es")
dataset = load_dataset("KRadim/edit_amazon_reviews_multi_es")
# veamos la distribución del dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'lenght_review_body', 'lenght_review_title', 'lenght_product_category'],
        num_rows: 199500
    })
    validation: Dataset({
        features: ['id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'lenght_review_body', 'lenght_review_title', 'lenght_product_category'],
        num_rows: 5250
    })
    test: Dataset({
        features: ['id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'lenght_review_body', 'lenght_review_title', 'lenght_product_category'],
        num_rows: 5250
    })
})

Ahora vamos a hacer una pequeña muestra aleatoria del conjunto de entrenamiento, para tener una idea de la forma que tienen los ejemplos. Podrás ver que hay un id de review, de producto, de reviewer, el número de estrellas (de menos a más satisfecho, de 1 a 5) y el contenido de la review. La idea va a ser intentar predecir el número de estrellas a partir de la review.

In [4]:
import random
import pandas as pd
from datasets import ClassLabel
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    "Está función está sacada de https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb"

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category
0,160420,4,"Tiene un tamaño muy bueno para una cocina pequeña. Calienta con rapidez y se apaga sola. La única objeción es que al ser metálica, hay que tener cuidado y tocar sólo los componentes de plástico porque te quemas.",Cumple su funcion,es,kitchen,211,17,7
1,146938,2,Necesita una recarga de muchas horas y luego se agota enseguida la batería. No puede usarse con el adaptador de corriente enchufado...,Dura muy poco,es,personal_care_appliances,134,13,24
2,99702,3,Está entretenido pero es un cuadernillo de pocas hojas y de tapa blanda. Nada que ver con los originales de Buscando a Wally.,Discreto,es,book,125,8,4
3,131720,2,"Cuando pones dos discos duro en la parte de usb se quedan sin reaccionar. Se calienta un poco, por lo demás bien",Dos disco duros a la vez no funcionan,es,pc,112,37,2
4,165595,2,Hola buenas me a llegado el patinete pero me hace falta el sillín que puedo hacer para que me llegue,El sillín por favor,es,sports,100,19,6
5,46137,1,"No se oye bien, ni si quiera de manera aceptable, la música por un lado y el sonido por otro. La mejor manera de tirar el dinero.",Para tirarlo,es,electronics,129,12,11
6,150612,3,"Hay que soltar la pestaña del todo para que a la siguiente presión salga el líquido. Acaba siendo agotador. Es una pena porque el spray de agua es un difusor, cuando a menudo ese tipo de producto tiene un spray que suelta un hilo de agua directo, lo cual tiene el mismo efecto.",Podría funcionar mejor,es,beauty,277,22,6
7,91415,1,"Compré hace seis meses uno, y en principio bien, aunque al poco de usarlo, se me despegó la cinta que sujeta el pie. Pero un día la pantalla no encendía, se me fue la pantalla de repente y no se puede cambiar la batería porque dice el mismo producto que la pantalla no se quita. El folleto de instrucciones es muy malo y por ejemplo dice cómo cambiar la batería, pero el mismo producto en la web dice que no se cambia. Tampoco vienen en español. Ahora he comprado otro, después de que se llevaran el anterior, y la sorpresa es que ha venido con la pantalla fuera, y las patas no encajan cuando se abren. Así que mi experiencia no es buena con este producto.",Malo,es,sports,657,4,6
8,39646,2,"Compacto, de buen material, lamentablemente no funcionan los enchufes de un lado, y únicamente los de USB si funcionan. Quise devolverlo pero paso el periodo de devolución, básicamente un error mío por no probarlo y ahora que estoy de viaje me dejó abandonado.",Decepción,es,pc,260,9,2
9,158306,2,"el transmisor en la sala y el receptor a 10 metros con dos paredes por medio y se quedaba la imagen congelada en la TV . Con el emisor y el receptor a un metro y conectado al proyector, la imagen es de una calidad muy pobre si la comparo con la señal que me dá con cable hdmi directo al proyector",Buena idea pero no funciona,es,electronics,296,27,11


Es muy fácil transformar un dataset a un dataframe de Pandas:

In [5]:
dataset.set_format("pandas")
df = dataset["train"][:]
df.head()

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category
0,111516,1,No llegaron las hélices,Mal servicio,es,electronics,23,12,11
1,107084,4,Me encanta lo ligera y manejable que es. Plega...,Manejable y ligera,es,baby_product,420,18,12
2,199386,1,"De las dos baterias , hay una que no funciona",No comprare mas,es,home_improvement,45,15,16
3,69470,2,"la iluminación es genial, pero el adhesivo pos...",el adhesivo no pega bien,es,home,105,24,4
4,114836,4,La he comprado porque por estética y colores e...,Original,es,home,165,8,4


In [6]:
df["product_category"].value_counts()

product_category
home                        26908
wireless                    25794
toy                         13614
sports                      13173
pc                          11144
home_improvement            10906
electronics                 10374
beauty                       7333
automotive                   7114
kitchen                      6673
apparel                      5706
drugstore                    5494
book                         5249
furniture                    5185
baby_product                 4841
office_product               4763
lawn_and_garden              4221
other                        3911
pet_products                 3707
personal_care_appliances     3529
luggage                      3340
camera                       3029
shoes                        2748
digital_ebook_purchase       1869
video_games                  1724
jewelry                      1601
musical_instruments          1533
watch                        1486
industrial_supplies          14

In [7]:
df["stars"].value_counts()

stars
4    39935
5    39927
2    39901
3    39889
1    39848
Name: count, dtype: int64

In [8]:
dataset.reset_format()

## 3. Fusionar las clasificaciones por estrellas

Para simplificar, vamos a quitar aquellas reseñas neutrales (3 estrellas), de esta forma conseguiremos que los ejemplos sean dicotómicos (negativo o positivo). Muchas veces la clave está en simplificar el problema.

In [9]:
# Una forma rápida es filtrar usando una función lambda (sin nombre).
dataset = dataset.filter(lambda x : x["stars"] != 3)

Para conseguir que un ejemplo sea positivo (1) o negativo (0), vamos a fusionar los ejemplos de 1 y 2 estrellas, y los de 4 y 5 estrellas.

In [10]:
def merge_star_ratings(examples):
    if examples["stars"] <= 2:
        label = 0
    else:
        label = 1
    return {"labels": label}

In [11]:
dataset = dataset.map(merge_star_ratings)

In [12]:
show_random_elements(dataset["train"], num_examples=3)

,id,stars,review_body,review_title,language,product_category,lenght_review_body,lenght_review_title,lenght_product_category,labels
0,127986,5,"La verdad es que cumple mejor de lo que esperaba su función. La duración de la batería, en este caso es de unas 12 horas. Posee una aplicación oficial, para Android e iOs, desde la cual se podrá controlar remotamente el gimbal vía Bluetooth, así como calibrarlo. Buen compra.",Funciona bien,es,camera,275,13,6,1
1,164242,5,"Los compré para el cumpleaños de mi novio y se escuchan muy bien para el precio que tienen (al principio los bajos no sonaban tan fuerte como me gusta, pero cambiando los ajustes de audio del móvil ahora suenan más potentes). Aguantan bien de batería, el estándar de este tipo de auriculares.",Buena calidad,es,electronics,292,13,11,1
2,145405,1,"No funciona ninguno ni el negro, ni el amarillo, ni el azul, ni el magenta y como ya estoy fuera de plazo no lo puedo devolver",No los reconoce la impresora,es,office_product,126,28,14,0


## 4. Tokenizar las reseñas

Como hemos visto hasta ahora, no se le pasa la frase al completo al modelo, sino que se parte en tokens. La tokenización de las reseñas se basa en un modelo pre-entrenado. Puedes seleccionar un modelo en el [Hugging Face Hub](https://huggingface.co/models) (puedes filtrar por la tarea o problema del modelo, lenguaje, etc.). En este caso, vamos a usar Roberta, un modelo de lenguaje en español entrenado en el BSC. Indicamos su nombre de checkpoint como se indica en 🤗 Hub, y descargamos el tokenizador asociado a ese modelo.

*Nota: la versión oficial no está disponible, por lo que usaremos una copia que existe en el repositorio*

In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

#model_checkpoint='PeterPanecillo/PlanTL-GOB-ES-roberta-base-bne-copy' # modelo ya no disponible
model_checkpoint="dccuchile/bert-base-spanish-wwm-uncased"  # usaremos esta alternativa
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Asignar el token de fin de frase como token de relleno (padding)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# IMPORTANTE: Re-instanciar el DataCollator después de este cambio
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Vamos a aplicar la tokenización a las reseñas. De momento vamos a jugar con un ejemplo.


In [15]:
# El tamaño del vocabulario del tokenizador
tokenizer.vocab_size

31002

En el siguiente ejemplo puedes ver algunos tokens especiales (como el comienzo `<s>` y fin `</s>`) y que algunas palabras, como lewis, al no ser común en español, se ha partido en dos tokens (esto lo hace el algoritmo intentando maximizar los tokens conocidos). Un token desconocido sería `<UNK>`.

In [16]:
# Un ejemplo del codificar y descodificar.
text = "¡hola, me llamo Lewis!"
tokenized_text = tokenizer.encode(text)

for token in tokenized_text:
    print(token, tokenizer.decode([token]))

4 [CLS]
1120 ¡
1734 hola
1019 ,
1094 me
5592 llamo
1165 le
1004 ##w
1056 ##is
1109 !
5 [SEP]


A continuación podemos visualizar los tensores obtenidos en PyTorch. Se pueden ver los ids dados a cada token.

In [17]:
encoded_text = tokenizer(text, return_tensors="pt")
encoded_text

{'input_ids': tensor([[   4, 1120, 1734, 1019, 1094, 5592, 1165, 1004, 1056, 1109,    5]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Vamos a definir una función que reciba un ejemplo, vaya a la columna del review, y aplique el tokenizador. Vamos a aplicar `truncation=true` para recortar los textos demasiados grandes. Recuerda, los modelos pre-entrenados traducen los textos de entrada a vectores de tamaño fijo, por lo que hay que cortar aquellos ejemplos que sean demasiado grandes.

In [18]:
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [19]:
columns = dataset["train"].column_names
columns.remove("labels")
# map es como apply de pandas.
# remove_columns es opcional, pero simplifica. Quitará las columnas que no queremos.
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 159611
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4190
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4199
    })
})

Lo siguiente es una muestra el input del modelo. Como el modelo es del tipo fill mask ("predice la palabra que falta"), hay que indicarle un attention mask. En este caso la entrada se reciba entera (todo 1). Los input_ids son los tokens y el label es la clasificación.

In [20]:
# Esto será el input del modelo
encoded_dataset["train"][0]

{'labels': 0,
 'input_ids': [4, 1054, 7175, 1085, 19613, 22534, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

## 5. Cargar el modelo preentrenado

Desde el módulo de modelos de secuencias (SequenceClassification), descargamos el modelo con el nombre que hemos indicado arriba (el pre-entrenado seleccionado del hub). Indicamos que queremos el modelo para hacer clasificación con dos etiquetas (esta última capa es la que entrenaremos).

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, 
        use_safetensors=True)   # esto es obligatorio si tienes una versión de pytorch < 2.6

# Asegurarnos de que el modelo también sepa cuál es el ID del token de padding
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


El warning que sale nos dice que el modelo se ha cargado con una capa de salida vacía, por lo que deberías entrenarlo. Si le pasamos el primer ejemplo en encoded_text, obtenemos los logits (lo que luego será 0 o 1).

In [22]:
outputs = model(**encoded_text)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[0.0978, 0.1877]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## 6. Definir las métricas de rendimiento

Definimos una métrica, usaremos accuracy, ya que tenemos un dataset balanceado.

In [23]:
import evaluate

# Ahora se usa evaluate.load
metric = evaluate.load("accuracy")

Esta función se usará para calcular la métrica durante el entrenamiento.

In [24]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## 7. Afinar el modelo preentrenado

Trainer te permite hacer cosas muy avanzadas. Incluso publicar el modelo una vez has acabado.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_review_classifier",
    eval_strategy="epoch",
    push_to_hub=False,             # No publicar en Huggingface
    save_strategy="no",            # No guardar checkpoints intermedios
    report_to="none",              # Sin reportes externos
    #num_train_epochs=3,           # Ajusta aquí el número de épocas
)

Aquí instanciamos el Trainer, indicando nuestro train dataset y la validación, además del tokenizer. Y finalmente entrenamos. Cuidado, esto requiere de una buena GPU. En una RTX3090 esta ejecución tardó 50 minutos. Puedes bajar el número de épocas arriba.

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator # <--- Indica el tokenizador
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.698000,0.692604,0.517028
2,0.695600,0.692572,0.517028
3,0.694400,0.723629,0.482972


TrainOutput(global_step=59856, training_loss=0.6318196128696338, metrics={'train_runtime': 2725.9737, 'train_samples_per_second': 175.656, 'train_steps_per_second': 21.958, 'total_flos': 2.060393537030334e+16, 'train_loss': 0.6318196128696338, 'epoch': 3.0})

Guardamos nuestro modelo en un directorio para nosotros. También lo podemos publicar en el hub (mira el final).

In [40]:
id2label = {0: "Negativo", 1: "Positivo"}
model.config.id2label = id2label

dir="amazon_review"
model.save_pretrained(save_directory=dir)

Vamos a usar un `pipeline`, que nos permite usar un modelo end-to-end, es decir, directamente pasarle texto y que nos indique la clasificación (automáticamente, sin tener que hacer los pasos de tokenizar).

In [41]:
from transformers import pipeline

pipe = pipeline(task="sentiment-analysis",
                model=dir,
                tokenizer=tokenizer)
pipe("¡me encanta el ipad!")

Device set to use cuda:0


[{'label': 'Positivo', 'score': 0.6063070893287659}]

In [42]:
pipe("El artículo no es lo que esperaba, es un mamotreto inútil que no sirve para nada")


[{'label': 'Positivo', 'score': 0.6063070893287659}]

In [43]:
# juega con el modelo

pipe("No me gusta nada")


[{'label': 'Positivo', 'score': 0.6063070893287659}]

In [48]:
import torch
def probar_frase(frase):
    inputs = tokenizer(frase, return_tensors="pt").to('cuda')
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    label = id2label[predicted_class_id]
    score = torch.softmax(logits, dim=1).max().item()
    print(f"Frase: '{frase}' -> {label} (Confianza: {score:.2f})")

In [49]:
probar_frase("Es una película horrible, perdí mi tiempo.")
probar_frase("Me ha encantado, es una obra maestra maravillosa.")

Frase: 'Es una película horrible, perdí mi tiempo.' -> Positivo (Confianza: 0.61)
Frase: 'Me ha encantado, es una obra maestra maravillosa.' -> Positivo (Confianza: 0.61)


## 8. Subida del modelo hacia el Hugging Face Hub

Para compartir tu modelo con la comunidad, primero crea una cuenta en el [Hugging Face Hub](https://huggingface.co/join). A continuación, ejecute la siguiente celda y proporcione su nombre de usuario y contraseña para generar un token de autenticación:

In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login

Si no tienes instalado [Git LFS](https://git-lfs.github.com), puedes hacerlo descomentando y ejecutando la celda de abajo:

In [ ]:
!apt install git-lfs
!git config --global user.email "lewis.c.tunstall@gmail.com"
!git config --global user.name "Lewis Tunstall"

Para más detalles sobre el envío de modelos al Hub, vea el siguiente vídeo:

In [ ]:
YouTubeVideo("A5IWIxsHLUw", width=600, height=400)

In [ ]:
trainer.push_to_hub()

A continuación, si hemos descomentado las líneas del Trainer que hemos dejado comentadas, y por tanto publicado nuestro modelo en el Hub, podemos volver a descargarlo y usarlo con el pipeline.



In [ ]:
from transformers import pipeline

model_checkpoint = "lewtun/roberta-base-bne-finetuned-amazon_reviews_multi"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

In [ ]:
pipe("¡me encanta el ipad!")